In [4]:
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key

In [16]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Import the WeatherPy_vacation.csv file as a new DataFrame
part_two_hotel_data_df = pd.read_csv(r'weather_data/WeatherPy_vacation.csv')
part_two_hotel_data_df

,City_ID,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
0,0,Port Alfred,ZA,69.01,-33.59,26.89,scattered clouds,The Halyards Hotel
1,1,Bluff,AU,74.39,-23.58,149.07,clear sky,Bluff Hotel
2,3,Mar Del Plata,AR,69.57,-46.43,-67.52,overcast clouds,Hotel Costanera Del Sur
3,7,Rikitea,PF,76.03,-23.12,-134.97,broken clouds,Pension Maro'i
4,8,Nuevo Casas Grandes,MX,63.88,30.42,-107.91,clear sky,Hotel Villa Colonial
...,...,...,...,...,...,...,...,...
208,545,Tha Bo,TH,66.20,17.85,102.58,clear sky,บ้านพักริมโขง รีสอร์ท
209,547,Natal,BR,82.40,-5.81,-35.21,scattered clouds,Monza Palace Hotel
210,548,Vicksburg,US,66.20,32.35,-90.88,clear sky,La Quinta Inn & Suites by Wyndham Vicksburg
211,552,Kota Kinabalu,MY,77.00,5.98,116.07,broken clouds,Gaya Centre Hotel


In [15]:
close_cities_df = part_two_hotel_data_df.loc[(part_two_hotel_data_df["City"] == "Nome")|(part_two_hotel_data_df["City"] == "Vicksburg")|(part_two_hotel_data_df["City"] == "Pineville")|(part_two_hotel_data_df["City"] == "Corpus Christi")|(part_two_hotel_data_df["City"] == "Matamoros")]
close_cities_df

,City_ID,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
61,145,Corpus Christi,US,77.00,27.80,-97.40,clear sky,Holiday Inn Corpus Christi Downtown Marina
88,217,Matamoros,US,77.00,25.87,-97.51,clear sky,Hotel Alameda Express
89,219,Pineville,US,71.01,31.32,-92.43,clear sky,"Country Inn & Suites by Radisson, Pineville, LA"
195,510,Nome,US,73.99,30.04,-94.42,clear sky,Silver Spur Mobile Home & RV Park
210,548,Vicksburg,US,66.20,32.35,-90.88,clear sky,La Quinta Inn & Suites by Wyndham Vicksburg


In [62]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = close_cities_df[["City", "Country", "Max Temp", "Lat", "Lng", "Current Description"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
61,Corpus Christi,US,77.00,27.80,-97.40,clear sky,
88,Matamoros,US,77.00,25.87,-97.51,clear sky,
89,Pineville,US,71.01,31.32,-92.43,clear sky,
195,Nome,US,73.99,30.04,-94.42,clear sky,
210,Vicksburg,US,66.20,32.35,-90.88,clear sky,


In [63]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [64]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")

In [68]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [69]:
#Typing out tuples is what it shows to do on thelink in the description
# Latitude-longitude pairs
corpus_christi = (27.8, -97.4)
matamoros = (25.87, -97.51)
pineville = (31.32, -92.43)
nome = (30.04, -94.42)
vicksburg = (32.35, -90.88)
fig = gmaps.figure(center=(29.7, -95.4), zoom_level=6)
fig.add_layer(gmaps.bicycling_layer())
fig.add_layer(gmaps.transit_layer())
fig.add_layer(gmaps.traffic_layer())
mata2vicks = gmaps.directions_layer(matamoros, vicksburg, waypoints=[corpus_christi, pineville, nome], travel_mode = 'DRIVING')
fig.add_layer(mata2vicks)
fig

Figure(layout=FigureLayout(height='420px'))

In [70]:
# Add markers
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(29.7, -95.4), zoom_level = 5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))